#### $$x^2 - \frac{e^x}{5} = 0$$

In [145]:
from math import *

ln = lambda x: log(x, e)

def f(x : float) -> float:
    return x**2 - exp(x) / 5

iterations = 10

def check_method(method, x0 : float):
    x = method(x0)
    print(f"x = {x}, ε = {abs(f(x))}")

$f(x) = x^2 - \frac{e^x}{5}$

Из геометрических соображений видно, что уравнение $f(x) = 0$ имеет 3 решения, по одному на каждом из интервалов: 

$I_1 = (-2; -0.1)$, $I_2 = (0.1; 2)$, $I_3 = (2; +\infty)$

Для их нахождения построим МПИ:

**1)** $e^x - 5x^2 = 0 \Leftrightarrow x = \ln(5x^2)$

$x_{n+1} = \varphi(x_n)$, где $\varphi(x) = \ln(5x^2)$

$|\varphi'(x)| = \frac{2}{|x|} \Rightarrow$ отображение является сжимающим на $I_3$.

**2)** $xe^x - 5x^3 = 0 \Leftrightarrow x = \sqrt[3]{\frac{xe^x}{5}}$

$x_{n+1} = \psi(x_n)$, где $\psi(x) = \sqrt[3]{\frac{xe^x}{5}}$

$|\psi'(x)| = \frac{|1+x|}{3}\sqrt[3]{\frac{e^x}{5x^2}}$

Анализ показывает, что $|\psi'(x)| < 1$ на $I_1$ и $I_2 \Rightarrow$ отображение является сжимающим на этих интервалах.

In [146]:
def find_solution_by_phi(x0 : float) -> float:
    x = x0
    for n in range(iterations):
        x = ln(5 * x * x)
    return x

def find_solution_by_psi(x0 : float) -> float:
    x = x0
    for n in range(iterations):
        x = cbrt(x * exp(x) / 5)
    return x

check_method(find_solution_by_phi, 10)
check_method(find_solution_by_psi, 1)
check_method(find_solution_by_psi, -1)

x = 4.708472854286686, ε = 0.006822681711739165
x = 0.6060105489732267, ε = 0.0006280426096200165
x = -0.3714177857271311, ε = 2.9301973680784243e-08


### Метод Ньютона

$f'(x) \approx \frac{f(x + \varepsilon_d) - f(x - \varepsilon_d)}{2\varepsilon_d}$

In [147]:
eps_d = 1e-5

def df(x):
    return (f(x + eps_d) - f(x - eps_d)) / (2 * eps_d)

def find_solution_by_Newton(x0 : float) -> float:
    x = x0
    for n in range(iterations):
        x = x - f(x) / df(x)
    return x

check_method(find_solution_by_Newton, 10)
check_method(find_solution_by_Newton, 1)
check_method(find_solution_by_Newton, -1)

x = 4.7079379182533705, ε = 1.5873702352564578e-09
x = 0.6052671213146186, ε = 1.1102230246251565e-16
x = -0.37141775245917386, ε = 0.0


#### $$ \begin{cases} tg(xy + 0.4) = x^2\\ 0.6x^2 + 2y^2 = 1 \end{cases} $$

In [148]:
from matrixlib import *

def f(u : Vector) -> Vector:
    x = u[0]
    y = u[1]
    return Vector(tan(x*y + 0.4) - x*x, 0.6*x*x + 2*y*y - 1)

iterations = 20
norm = get_Euclidian_norm

def check_method(method, u0 : Vector):
    x = method(u0)
    print(f"x = ({x[0]}, {x[1]})ᵀ, ε = {norm(f(x))}")

Второе уравнение задаёт эллипс с полуосями $a = \sqrt{\frac{5}{3}}, b = \frac{1}{\sqrt{2}}$.

Тогда решения должны принадлежать множеству $[-\sqrt{\frac{5}{3}}, \sqrt{\frac{5}{3}}]\times[-\frac{1}{\sqrt{2}}, \frac{1}{\sqrt{2}}]$.

In [149]:
def test(u0 : Vector) -> Vector:
    x = u0[0]
    y = u0[1]
    for n in range(iterations):
        x = sqrt(tan(x*y + 0.4))
        y = sqrt(0.5 - 0.3*x*x)
    return Vector(x, y)

check_method(test, Vector(1, 0.6))

x = (1.0484001359003647, 0.4126222806795933)ᵀ, ε = 1.5348094128952994e-08


#### Метод Ньютона

In [150]:
eps_d = 1e-7 #differentiation accuracy
eps_l = 1e-7 #linear system solving accuracy

def solve_linear_system_by_Seidel(A : Matrix, f : Vector, x0 : Vector) -> Vector:
    f = A.T() * f
    A = A.T() * A
    n = A.n
    x = x0.copy()
    x_dual = Vector.zeros(n)
    while norm(A * x - f) > eps_l:
        for i in range(n):
            x_dual[i] = -(sum([A[i, j] * x_dual[j] for j in range(i) if i != j]) 
                          + sum([A[i, j] * x[j] for j in range(i+1, n) if i != j]) 
                          - f[i]) / A[i, i]
        x, x_dual = x_dual, x
    return x

def J(u : Vector) -> Matrix:
    dfx = (f(u + Vector(eps_d, 0)) - f(u - Vector(eps_d, 0))) / (2 * eps_d)
    dfy = (f(u + Vector(0, eps_d)) - f(u - Vector(0, eps_d))) / (2 * eps_d)
    return Matrix([(dfx[0], dfy[0]), (dfx[1], dfy[1])])

def find_solution_by_Newton(u0 : Vector) -> Vector:
    u = u0.copy()
    for n in range(iterations):
        du = solve_linear_system_by_Seidel(J(u), -f(u), Vector(0, 0))
        u += du
    return u

check_method(find_solution_by_Newton, Vector(1, 1))
check_method(find_solution_by_Newton, Vector(1, -1))
check_method(find_solution_by_Newton, Vector(-1, 1))
check_method(find_solution_by_Newton, Vector(-1, -1))

x = (1.0484001675053962, 0.41262227894858106)ᵀ, ε = 4.522026215295088e-08
x = (0.3798484301636774, -0.6758065939361407)ᵀ, ε = 1.939304354293348e-08
x = (-0.3798484301636774, 0.6758065939361407)ᵀ, ε = 1.939304354293348e-08
x = (-1.0484001675053962, -0.41262227894858106)ᵀ, ε = 4.522026215295088e-08
